# Evaluation of study results

In [ ]:
import numpy as np
import geopandas as gpd
import folium
import pandas as pd

In [ ]:
MAX_PLZ_STELLEN = 3

In [ ]:
data = gpd.read_file("data/Umfragedaten.csv")
plz_gebiete = gpd.read_file("data/georef-germany-postleitzahl.geojson")

In [ ]:
shortened_plz = plz_gebiete["name"].apply(lambda x: x[:MAX_PLZ_STELLEN])
plz_gebiete["shortened_plz"] = shortened_plz
shortened_plz = data["Was ist Ihre Postleitzahl?"].apply(lambda x: x[:MAX_PLZ_STELLEN])
data["shortened_plz"] = shortened_plz

In [ ]:
short_plzs = []
combined_polygons = []
for _, short_plz in enumerate(plz_gebiete["shortened_plz"].unique()):
    all_areas = plz_gebiete.query("`shortened_plz` == @short_plz")
    polygons = all_areas["geometry"].union_all()
    short_plzs.append(short_plz)
    combined_polygons.append(polygons)

new_plz_gebiete = gpd.GeoDataFrame.from_dict({"shortened_plz": short_plzs, "geometry": combined_polygons}, crs="EPSG:4326")

In [ ]:
merged_data = pd.merge(data, new_plz_gebiete, on="shortened_plz")
merged_geo_df = gpd.GeoDataFrame(merged_data, geometry="geometry")
prevelance = merged_geo_df["shortened_plz"].value_counts()

In [ ]:
merged_geo_df["shortened_plz"]

In [ ]:
merged_geo_df["prevelance"] = merged_geo_df["shortened_plz"].apply(lambda x: prevelance[f"{x}"])

In [ ]:
m = folium.Map()

In [ ]:
folium.GeoJson(merged_geo_df, style_function=lambda row: {
    "fillColor": "#00ff00" if row["properties"]["prevelance"] > 10 else "#ff00ff"
}).add_to(m)

In [ ]:
m